In [ ]:
import tensorflow as tf
from tensorflow import keras

# Load the model from .keras format
model = keras.models.load_model('Downloads/model_2024_hairstyle.keras')

# Export the model in SavedModel format
model.export('Downloads/model_2024_hairstyle_savedmodel')

# Convert the model from the SavedModel directory to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('Downloads/model_2024_hairstyle_savedmodel')
tflite_model = converter.convert()

# Save the converted TFLite model
with open('Downloads/hairstyle-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

print("Model successfully converted to TensorFlow Lite.")


In [ ]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="Downloads/hairstyle-model.tflite")

# Allocate tensors (required before accessing details)
interpreter.allocate_tensors()

# Get the output tensor details
output_details = interpreter.get_output_details()

# Display the output index
for output in output_details:
    print(f"Output Name: {output['name']}, Index: {output['index']}")

In [ ]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [ ]:
import numpy as np

img_url = 'https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg'


img_d = download_image(img_url)

img_d_p = prepare_image(img_d, target_size=(299, 299))

img_array = np.array(img_d_p)

img_array = img_array / 255.0

# Step 5: Extract the first pixel's R channel value
first_pixel_r_channel = img_array[0, 0, 0]

print(f"The value of the first pixel in the R channel is: {first_pixel_r_channel}")

In [ ]:
import numpy as np
import tensorflow as tf
from io import BytesIO
from urllib import request
from PIL import Image

# Functions to download and prepare the image
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# Download and preprocess the image
url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
target_size = (200, 200)

# Step 1: Download and prepare the image
img = download_image(url)
img = prepare_image(img, target_size)

# Step 2: Convert the image to a numpy array and preprocess
img_array = np.array(img) / 255.0  # Normalize pixel values to [0, 1]
img_array = img_array.astype(np.float32)  # Explicitly cast to FLOAT32
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Step 3: Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="Downloads/hairstyle-model.tflite")
interpreter.allocate_tensors()

# Step 4: Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Step 5: Set the input tensor
interpreter.set_tensor(input_details[0]['index'], img_array)

# Step 6: Run the inference
interpreter.invoke()

# Step 7: Get the output
output = interpreter.get_tensor(output_details[0]['index'])
print(f"Model output: {output[0][0]}")